In [ ]:
# mlflow_experiment.ipynb

import mlflow
import mlflow.pyfunc
from transformers import MarianMTModel, MarianTokenizer

# Charger le modèle
model_name = "Helsinki-NLP/opus-mt-ja-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Wrapper pour MLflow
class TranslationModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        self.tokenizer = tokenizer
        self.model = model

    def predict(self, context, model_input):
        inputs = self.tokenizer([model_input], return_tensors="pt", truncation=True, padding=True)
        outputs = self.model.generate(**inputs)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

# Dossier de sauvegarde local
mlflow.set_experiment("translation-ja-en")

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=TranslationModel()
    )
